In [8]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date, timedelta, datetime
import time

# create app
sc = SparkSession.builder.appName("cleansingcrimes")\
.config ("spark.sql.shuffle.partitions", "50")\
.config("spark.driver.maxResultSize","5g")\
.config ("spark.sql.execution.arrow.enabled", "true")\
.getOrCreate()

# Extract Json file
df = sc.read.csv('dataset/crimes.csv') 
df.show(5)

+----+--------+-------+----------------+----------------+---------+---------+--------------------+---------+---------------+--------------------+--------------------+-------+
| _c0|     _c1|    _c2|             _c3|             _c4|      _c5|      _c6|                 _c7|      _c8|            _c9|                _c10|                _c11|   _c12|
+----+--------+-------+----------------+----------------+---------+---------+--------------------+---------+---------------+--------------------+--------------------+-------+
|null|Crime.ID|  Month|     Reported.by|    Falls.within|Longitude| Latitude|            Location|LSOA.code|      LSOA.name|          Crime.type|Last.outcome.cate...|Context|
|   1|    null|2016-04|Cleveland Police|Cleveland Police|-1.238154|54.710767|On or near Greyfr...|E01011954|Hartlepool 001A|Anti-social behav...|                null|     NA|
|   2|    null|2016-04|Cleveland Police|Cleveland Police|-1.239958|54.710607|On or near Marsha...|E01011954|Hartlepool 001A|A

In [9]:
# rename column
df = df.na.fill(value="No",subset=["_c0"])
row_list = df.select('*').collect()
list_column = [ x for x in row_list[0]]
df = df.toDF(*list_column)
# drop first row
df = df.filter(df.No != "No")
df.show(5)

+---+--------+-------+----------------+----------------+---------+---------+--------------------+---------+---------------+--------------------+---------------------+-------+
| No|Crime.ID|  Month|     Reported.by|    Falls.within|Longitude| Latitude|            Location|LSOA.code|      LSOA.name|          Crime.type|Last.outcome.category|Context|
+---+--------+-------+----------------+----------------+---------+---------+--------------------+---------+---------------+--------------------+---------------------+-------+
|  1|    null|2016-04|Cleveland Police|Cleveland Police|-1.238154|54.710767|On or near Greyfr...|E01011954|Hartlepool 001A|Anti-social behav...|                 null|     NA|
|  2|    null|2016-04|Cleveland Police|Cleveland Police|-1.239958|54.710607|On or near Marsha...|E01011954|Hartlepool 001A|Anti-social behav...|                 null|     NA|
|  3|    null|2016-04|Cleveland Police|Cleveland Police|-1.239958|54.710607|On or near Marsha...|E01011954|Hartlepool 001A|An